<span style="font-size:3em; text-align:center">Information System Security -</span>

<span style="font-size:3em; text-align:center">Anomaly-based Intrusion Detection System</span>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data is contained in 8 different CSV files, each containing different attack data at different times. So first thing we must do is merge all the data from files into one pandas DataFrame.

In [ ]:
import pandas as pd
import glob

In [ ]:
# Saving all .csv files in folder to list.
path = "/content/drive/MyDrive/CSVs"
files = [file for file in glob.glob(path + "**/*.csv", recursive=True)]

In [ ]:
[print(f) for f in files]

/content/drive/MyDrive/CSVs/Friday-WorkingHours-Morning.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Tuesday-WorkingHours.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Monday-WorkingHours.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Wednesday-workingHours.pcap_ISCX.csv
/content/drive/MyDrive/CSVs/Copy of Monday-WorkingHours.pcap_ISCX.csv


[None, None, None, None, None, None, None, None, None]

In [ ]:
# Reading all the csv files into dataframes and putting thoose DFs to one list.

dataset = [pd.read_csv(f,  encoding='latin1') for f in files]

<ipython-input-20-6fd501f313bf>:3: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = [pd.read_csv(f,  encoding='latin1') for f in files]
<ipython-input-20-6fd501f313bf>:3: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = [pd.read_csv(f,  encoding='latin1') for f in files]


In [ ]:
# Here we can see the number of rows and columns for each table.

for d in dataset:
    print(d.shape)

(191033, 85)
(288602, 85)
(458968, 85)
(286467, 85)
(225745, 86)
(445909, 85)
(529918, 85)
(692703, 85)
(529918, 85)


In [ ]:
# We already established that all tables have the same number of columns, but are they the same columns?
# This next piece of code loops over all given tables and compares each of them to all others.

for i in range(0,len(dataset)):
  if i != len(dataset)-1:
    same_columns = dataset[i].columns == dataset[i+1].columns

    if False in same_columns:
      print(i)
      break

same_columns

ValueError: Lengths must match to compare

In [ ]:
# Combining all tables into one dataset. This is possilbe since all tables have the same columns,
# as we checked in the cell above.

dataset = pd.concat([d for d in dataset]).drop_duplicates(keep=False)
dataset.reset_index(drop=True, inplace = True)

In [ ]:
# By checking the shape of dataset we can confirm that concatenation has been successfull.

dataset.shape

# Preliminary data analysis

Some general info about the dataset. It contains roughly 2.5 million records across 79 columns. Data consists of mostly int64 and float64 types, except 3 attributes of 'object' type.

Dataset contains of network traffic data during different attacks, represented with values like: port numbers, IP adressses, packet lenghts, SYN/ACK/FIN/.. flag counts, packet size and other...

In [ ]:
#dataset = pd.read_csv('Dataset_clean.csv', index_col=[0])
dataset.info()

In [ ]:
dataset.describe()

Upon further inspection we can see that dataset contains 15 labels. Labels represent network/web attacks and BENIGN state which is the network traffic during normal business day.

In [ ]:
# Dataset conatains 15 labels.
#print(dataset[' Label'].unique())
#len(dataset[' Label'].unique())

print(dataset[' Label'].unique())
len(dataset[' Label'].unique())

In [ ]:
dataset.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

Most records in the dataset are of DDos and DOS Hulk attacks. This might pose a problem later in model training, considering that there is a very small amount of data for most attacks. Model selection will be greatly influenced by this information.

In [ ]:
data = dataset[' Label'].where(dataset[' Label'] != "BENIGN")

In [ ]:
plt.figure(figsize=(15,6))
chart = sns.countplot(data, palette="Set1")
plt.xticks(rotation=45, horizontalalignment="right")

# Data Cleaning

This chapter contains data cleaning code. We go through the process of renaming columns, removing NaN and non-finite values (-inf, inf) to get the data ready for visualization and model training.

## Renaming columns

In [ ]:
# Removing whitespaces in column names.

col_names = [col.replace(' ', '') for col in dataset.columns]
dataset.columns = col_names
dataset.head()

In [ ]:
# Here we can see that 'Label' column contains some wierd characters.

dataset["Label"].unique()

In [ ]:
# This next snippet uses regular expressions to replace wierd characters with dunders.

label_names = dataset['Label'].unique()


import re

label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]

label_names, len(label_names)

In [ ]:
# Replacing 'Label' column values with new readable values.

labels = dataset['Label'].unique()

for i in range(0,len(label_names)):
    dataset['Label'] = dataset['Label'].replace({labels[i] : label_names[i]})

dataset['Label'].unique()

In [ ]:
len(dataset['Label'].unique())

In [ ]:
 #Saving cleaned dataset.

dataset.to_csv("Dataset_cleaned.csv", index=False)

## Removing NULL values

In [ ]:
#dataset = pd.read_csv("Dataset_clean.csv", index_col=0)
dataset.head()

In [ ]:
# Checking if there are any NULL values in the dataset.

dataset.isnull().values.any()

In [ ]:
# Checking which column/s contain NULL values.

[col for col in dataset if dataset[col].isnull().values.any()]

In [ ]:
# Checking how many NULL values it this column contains.

dataset['FlowBytes/s'].isnull().sum()

In [ ]:
# Considering that only 334 rows contain NULL vlaues in the entire dataset, which makes about 0.01%, we
# can safely remove all NULL rows without spoiling the data.

334/dataset.shape[0]*100

In [ ]:
# Removing rows that contain NULL values and checking if number of removed rows is equal to the number of null values.

before = dataset.shape

dataset.dropna(inplace=True)

after = dataset.shape

before[0] - after[0]

In [ ]:
dataset.isnull().any().any()

## Removing *non-finite* values

In [ ]:
import numpy as np

In [ ]:
labl = dataset['Label']
dataset = dataset.loc[:, dataset.columns != 'Label'].astype('float64')

In [ ]:
# Checking if all values are finite.

np.all(np.isfinite(dataset))

In [ ]:
# Checking what column/s contain non-finite values.

nonfinite = [col for col in dataset if not np.all(np.isfinite(dataset[col]))]

nonfinite

In [ ]:
# Checking how many non-finite values each column contains.

finite = np.isfinite(dataset['FlowBytes/s']).sum()

dataset.shape[0] - finite

In [ ]:
# Checking how many non-finite values each column contains.

finite = np.isfinite(dataset['FlowPackets/s']).sum()

dataset.shape[0] - finite

In [ ]:
# Same as before, since there is a small number of non-finite values we can safely remove them from the dataset
# without spoiling the dataset.

# Replacing infinite values with NaN values.
dataset = dataset.replace([np.inf, -np.inf], np.nan)

In [ ]:
# We can see that now we have Nan values again.

np.any(np.isnan(dataset))

In [ ]:
# Bringing the Labels back into the dataset before deliting Nan rows.

dataset = dataset.merge(labl, how='outer', left_index=True, right_index=True)

In [ ]:
# Removing new NaN values.

dataset.dropna(inplace=True)

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
# Saving cleaned dataset.

dataset.to_csv("Dataset_clean_dropna.csv", index=False)

# Data visualization

So, by now we know our dataset has 78 features and is split into 15 categories (14 attacks and 1 "normal" state).
Next step is to try and visualize what the dataset looks like in feature space.
For this we will use principal component analysis (PCA) to reduce dimensionality and then pass the reduced dataset to t-SNE (t - Distributed Stohastic Neighbor Entities) for visual representation in 2D space.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline

In [ ]:
# We are going to pick 10.000 random rows from the dataset for visualization purposes.
# Setting the random seed for reproducability of results.

np.random.seed(42)

rand_perm = np.random.permutation(dataset.shape[0])
feature_cols = dataset.columns[:-1]

dataset_subset = dataset.loc[rand_perm[:10000],:]

In [ ]:
# Use iloc to select the rows based on integer-location indexing
np.random.seed(42)
rand_perm = np.random.permutation(dataset.shape[0])

dataset_subset = dataset.iloc[rand_perm[:10000], :]


In [ ]:
# We are going to pick 10.000 random rows from the dataset for visualization purposes.
# Setting the random seed for reproducability of results.

import numpy as np

np.random.seed(42)

# Instead of using permutation of the dataset shape, use the dataset index
rand_perm = np.random.choice(dataset.index, size=10000, replace=False)
feature_cols = dataset.columns[:-1]

dataset_subset = dataset.loc[rand_perm,:]

In [ ]:
dataset_subset = dataset_subset.replace([np.inf, -np.inf], np.nan)
dataset_subset.dropna(inplace=True)

In [ ]:
data_subset = dataset_subset[feature_cols].values

In [ ]:
# Performing the principal component analysis. With just 19 components the variance ratio remains 99%, which is great.

pca = PCA(n_components=19)
pca_res = pca.fit_transform(data_subset)

data_subset = None
np.sum(pca.explained_variance_ratio_)

In [ ]:
# Computing t-SNE.

tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=1000)
tsne_res = tsne.fit_transform(data_subset)
print("done")

In [ ]:
# Use the PCA results for t-SNE
tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=1000)
tsne_res = tsne.fit_transform(pca_res)
print("done")


In [ ]:
dataset_subset['tsne_firstD'] = tsne_res[:,0]
dataset_subset['tsne_secondD'] = tsne_res[:,1]

In [ ]:
plt.figure(figsize=(16,16))

sns.scatterplot(
    x="tsne_firstD", y="tsne_secondD",
    palette=sns.color_palette("hls", colors),
    data=dataset_subset,
    hue="Label",
    legend="full",
    alpha=0.3
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'dataset_subset' and 'tsne_res' are defined

# Calculate the number of unique labels for the color palette
colors = len(dataset_subset['Label'].unique())

plt.figure(figsize=(16, 16))

sns.scatterplot(
    x="tsne_firstD", y="tsne_secondD",
    palette=sns.color_palette("hls", colors), # Use the calculated 'colors' value here
    data=dataset_subset,
    hue="Label",
    legend="full",
    alpha=0.3
)

From the cell above we can see distribution of the data in 2D space. It is obvious that attacks are not spatialy well separated from normal state. Clusters of attacks can hardly be seen, instead they are found in the same place as the "normal state" datatpoints.

This insight leads us to conclude that the ML model will probably have some issues with this kind of data. ML model will have to be chosen with this in mind.

# Data preparation

In this chapter, final data preparation steps are taken before we use the data for model traning and testing.

These steps include:

* Data scaling
* Label encoding
* Data splitting

In [ ]:
#dataset = pd.read_csv("Dataset_clean_dropna.csv")

In [ ]:
import pandas as pd

# Step 1: Load the Full Dataset
# Replace 'path_to_your_csv_file.csv' with the actual path to your large dataset
file_path = '/content/drive/MyDrive/Dataset_clean_dropna.csv'
df = pd.read_csv(file_path)
print("Loaded full dataset with shape:", df.shape)

# Step 2: Sample a Subset of the Data
# Specify the sample fraction (e.g., 0.1 for 10% of the data)
sample_fraction = 0.2  # Adjust this fraction to reduce dataset size further if needed

# Randomly sample the data
df_sample = df.sample(frac=sample_fraction, random_state=42)
print("Sampled subset shape:", df_sample.shape)

# Step 3: Save the Sampled Subset as a New CSV File
output_path = 'small_cicids_sample.csv'
df_sample.to_csv(output_path, index=False)
print(f"Sampled dataset saved as {output_path}")

## Scaling the data

The next few cells contain the code for scaling the data into the size adequate for the ML algorithm.

In [ ]:
# ... (Your existing code for data cleaning and visualization) ...

# --- Insert the data sampling code here ---
import pandas as pd
cleaned_dataset = pd.read_csv("/content/drive/MyDrive/Dataset_clean_dropna.csv")
sample_dataset = cleaned_dataset.sample(frac=0.2, random_state=42)

# --- Replace the following line: ---
# dataset = pd.read_csv("Dataset_clean_dropna.csv")
# --- with this line to use the sample: ---
dataset = sample_dataset

# ... (The rest of your existing code for data preparation, model training, etc.) ...

In [ ]:
features.head()

In [ ]:
# For scaling the data, we use RobustScaler class from sklearn.

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

For scaling the data we used RobustScaler class from sklearn. RobustScaler is used to perserve outliers in the data.

In [ ]:
scaler = RobustScaler()
scaler.fit(features)

features = scaler.transform(features)

In [ ]:
# Checking if scaling has been succesful.
features[0]

## Label encoding

Label encoding is done when dataset contains categorical values (ex. 0-5, A/B/C, 55+). It is used to turn categorical values into numerical values by replacing data categories with integers starting with 0.

In [ ]:
# No need to do previous operations, just load clean saved dataset.

#dataset = pd.read_csv('Dataset_clean.csv', index_col=[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder

'Lables' column contains categorical values - 15 of them (14 types of attacks in our dataset +  1 normal state).

To convert this into numerical values we will use 'LabelEncoder' class from sklearn.

In [ ]:
LE = LabelEncoder()

LE.fit(labels)
labels = LE.transform(labels)

In [ ]:
# Labels have been replaced with integers.

np.unique(labels)

In [ ]:
# Checking that encoding reversal works.

d = LE.inverse_transform(labels)
d = pd.Series(d)
d.unique()

## Splitting the data

Final step to data preparation is splitting the data into traning and testing sets. For this there already exists _sklearn_ function that does all the splitting for us. This step is important so we can have representative data for evaluating our model. Both train and test samples should contain similar data variance.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# The next step is to split training and testing data. For this we will use sklearn function train_test_split().

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=.2)

features_train.shape, features_test.shape, labels_train.shape, labels_test.shape

In [ ]:
# Clearing variables.

dataset = None
finite = None
labl = None
d = None
features = None
labels = None

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Isolation Forest
from sklearn.ensemble import IsolationForest

print("Training Isolation Forest...")
iso_forest = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
iso_forest.fit(features_train)
iso_forest_pred = iso_forest.predict(features_test)
# Convert predictions from {-1, 1} to {0, 1}
iso_forest_pred = np.where(iso_forest_pred == -1, 1, 0)

# 2. One-Class SVM
from sklearn.svm import OneClassSVM

print("Training One-Class SVM...")
ocsvm = OneClassSVM(kernel='rbf', nu=0.1)
ocsvm.fit(features_train)
ocsvm_pred = ocsvm.predict(features_test)
# Convert predictions
ocsvm_pred = np.where(ocsvm_pred == -1, 1, 0)

# 3. Random Forest
from sklearn.ensemble import RandomForestClassifier

print("Training Random Forest...")
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(features_train, labels_train)
rf_pred = rf_classifier.predict(features_test)

# 4. Ensemble Approach
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

print("Training Ensemble...")
# Initialize base classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
svm = SVC(kernel='rbf', probability=True, random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# Create and train ensemble
ensemble = VotingClassifier(
    estimators=[('rf', rf), ('svm', svm), ('dt', dt)],
    voting='soft'
)
ensemble.fit(features_train, labels_train)
ensemble_pred = ensemble.predict(features_test)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred):.4f}")

# Evaluate all models
evaluate_model(labels_test, iso_forest_pred, "Isolation Forest")
evaluate_model(labels_test, ocsvm_pred, "One-Class SVM")
evaluate_model(labels_test, rf_pred, "Random Forest")
evaluate_model(labels_test, ensemble_pred, "Ensemble")

# Confusion Matrix Visualization
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plot confusion matrices
plot_confusion_matrix(labels_test, iso_forest_pred, 'Isolation Forest Confusion Matrix')
plot_confusion_matrix(labels_test, ocsvm_pred, 'One-Class SVM Confusion Matrix')
plot_confusion_matrix(labels_test, rf_pred, 'Random Forest Confusion Matrix')
plot_confusion_matrix(labels_test, ensemble_pred, 'Ensemble Confusion Matrix')

# ROC Curves
from sklearn.metrics import roc_curve, auc

def plot_roc_curves(models_dict, X_test, y_test):
    plt.figure(figsize=(10, 8))
    for name, model in models_dict.items():
        if hasattr(model, "predict_proba"):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            # For models without predict_proba
            y_pred = model.predict(X_test)
            y_pred_proba = np.where(y_pred == -1, 1, 0)

        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves Comparison')
    plt.legend()
    plt.show()

# Create dictionary of models for ROC curve comparison
models_dict = {
    'Random Forest': rf_classifier,
    'Ensemble': ensemble
}

# Plot ROC curves (only for models with predict_proba)
plot_roc_curves(models_dict, features_test, labels_test)

# Save the best performing model
import joblib

# Assuming Random Forest performs best (you can change this based on results)
best_model = rf_classifier
joblib.dump(best_model, 'best_model.joblib')
print("\nBest model saved as 'best_model.joblib'")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Import models
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# 1. Isolation Forest
print("Training Isolation Forest...")
iso_forest = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
iso_forest.fit(features_train)
iso_forest_pred = iso_forest.predict(features_test)
# Convert predictions from {-1, 1} to {0, 1}
iso_forest_pred = np.where(iso_forest_pred == -1, 1, 0)

# 2. Random Forest
print("Training Random Forest...")
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(features_train, labels_train)
rf_pred = rf_classifier.predict(features_test)

# 3. Decision Tree
print("Training Decision Tree...")
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(features_train, labels_train)
dt_pred = dt_classifier.predict(features_test)

# 4. K-Nearest Neighbors (KNN)
print("Training K-Nearest Neighbors...")
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(features_train, labels_train)
knn_pred = knn_classifier.predict(features_test)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred):.4f}")

# Evaluate all models
evaluate_model(labels_test, iso_forest_pred, "Isolation Forest")
evaluate_model(labels_test, rf_pred, "Random Forest")
evaluate_model(labels_test, dt_pred, "Decision Tree")
evaluate_model(labels_test, knn_pred, "K-Nearest Neighbors")

# Confusion Matrix Visualization
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plot confusion matrices
plot_confusion_matrix(labels_test, iso_forest_pred, 'Isolation Forest Confusion Matrix')
plot_confusion_matrix(labels_test, rf_pred, 'Random Forest Confusion Matrix')
plot_confusion_matrix(labels_test, dt_pred, 'Decision Tree Confusion Matrix')
plot_confusion_matrix(labels_test, knn_pred, 'K-Nearest Neighbors Confusion Matrix')

# Save the best performing model
import joblib

# Assuming Random Forest performs best (you can change this based on results)
best_model = rf_classifier
joblib.dump(best_model, 'best_model.joblib')
print("\nBest model saved as 'best_model.joblib'")


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
import joblib

# 1. Isolation Forest
print("Training Isolation Forest...")
iso_forest = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
iso_forest.fit(features_train)
iso_forest_pred = iso_forest.predict(features_test)

# Convert predictions from {-1, 1} to {0, 1} for evaluation
iso_forest_pred = np.where(iso_forest_pred == -1, 1, 0)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    # Changed 'average' to 'weighted' for multiclass support
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
# Evaluate Isolation Forest model
evaluate_model(labels_test, iso_forest_pred, "Isolation Forest")

# Confusion Matrix Visualization
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plot confusion matrix for Isolation Forest
plot_confusion_matrix(labels_test, iso_forest_pred, 'Isolation Forest Confusion Matrix')

# Save the Isolation Forest model
joblib.dump(iso_forest, 'isolation_forest_model.joblib')
print("\nIsolation Forest model saved as 'isolation_forest_model.joblib'")


In [ ]:
# Summary of Anomaly Detection Results
num_anomalies = np.sum(iso_forest_pred)
num_normal = len(iso_forest_pred) - num_anomalies
print(f"\nAnomaly Detection Summary:")
print(f"Total data points: {len(iso_forest_pred)}")
print(f"Detected anomalies: {num_anomalies}")
print(f"Normal data points: {num_normal}")

# Show index and predictions for anomalies
anomaly_indexes = np.where(iso_forest_pred == 1)[0]
print("\nIndexes of detected anomalies:", anomaly_indexes)

# Display the feature values of a few anomalies for inspection
print("\nExample of anomalous data points:")
# Use array indexing instead of .iloc
print(features_test[anomaly_indexes[:5]])  # Shows the first 5 anomalies

# Optional: Scatter plot (if you have two key features for visualization)
plt.figure(figsize=(10, 6))
# Assuming features_test has two features for simplicity; otherwise, choose two features.
# Use array indexing instead of .iloc
plt.scatter(features_test[:, 0], features_test[:, 1], c=iso_forest_pred, cmap='coolwarm', label='Anomaly Detection')
plt.title("Anomaly Detection Scatter Plot")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.legend(["Normal", "Anomaly"])
plt.show()

In [ ]:
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming features_train and features_test are your training and testing features
# Assuming labels_test is your ground truth labels for the test set

print("Training Isolation Forest for Anomaly Detection...")
iso_forest = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
iso_forest.fit(features_train)

# Predicting anomalies
iso_forest_pred = iso_forest.predict(features_test)
# Convert predictions from {-1, 1} to {0, 1} where 1 represents anomalies
iso_forest_pred = np.where(iso_forest_pred == -1, 1, 0)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    # Changed 'average' to 'weighted' for multiclass support
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=1):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
# Evaluate Isolation Forest
evaluate_model(labels_test, iso_forest_pred, "Isolation Forest")

# Summary of anomaly detection results
total_data_points = len(features_test)
detected_anomalies = np.sum(iso_forest_pred)
normal_data_points = total_data_points - detected_anomalies

print("\nAnomaly Detection Summary:")
print(f"Total test data points: {total_data_points}")
print(f"Detected anomalies (potential intrusions): {detected_anomalies}")
print(f"Normal data points: {normal_data_points}")

# Get indexes of detected anomalies for further inspection
anomaly_indexes = np.where(iso_forest_pred == 1)[0]  # Indexes where anomalies are detected
print("\nIndexes of detected anomalies:", anomaly_indexes)

# Optional: Display some feature values for a few detected anomalies for inspection
import pandas as pd
features_test_df = pd.DataFrame(features_test)  # Convert to DataFrame for easier viewing
anomalous_data = features_test_df.iloc[anomaly_indexes]
print("\nExample of anomalous data points:")
print(anomalous_data.head())  # Show details of the first few anomalies

# Confusion Matrix Visualization
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plot confusion matrix for Isolation Forest
plot_confusion_matrix(labels_test, iso_forest_pred, 'Isolation Forest Confusion Matrix')


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model on the training set
lr_model.fit(features_train, labels_train)

# Make predictions on the test set
lr_predictions = lr_model.predict(features_test)

# Evaluate the model
lr_accuracy = accuracy_score(labels_test, lr_predictions)
lr_precision = precision_score(labels_test, lr_predictions, zero_division=1)
lr_recall = recall_score(labels_test, lr_predictions)
lr_f1 = f1_score(labels_test, lr_predictions)

print("Logistic Regression Model Results:")
print(f"Accuracy: {lr_accuracy:.4f}")
print(f"Precision: {lr_precision:.4f}")
print(f"Recall: {lr_recall:.4f}")
print(f"F1-Score: {lr_f1:.4f}")


# Model training

For completing this task we chose to use a neural network. Specifically, the multi-layer perceptron, more specifically, feedforward neural network multi-class classifier with backpropagating algorithm. NN will be used to classifiy 14 different attacks and 1 normal state, as we saw from the labels in previous chapters.

In this chapter we go by explaning parts of the network and its hyperparameters.

In [ ]:
import tensorflow as tf
import datetime

#%load_ext tensorboard

Our tensorflow Sequential model has 3 layers. Input, 1 hidden and an output layer.

* Input layer has 78 neurons, one for each feature.
* Hidden layer has 67 neurons, this number has been calculated by [formula](https://www.heatonresearch.com/2017/06/01/hidden-layers.html) 2/3 the number of input neurons + number of output neurons.
* Output layer has 15 neurons, one for each class we predict.

For activation functions, we used standard functions for multi-class classification tasks - ReLu for hidden layer and _softmax_ function for output layer.

Finally, we use Dropout parameter set to 0.2 for randomly shutting off 20% of neurons in each learning iteration. This technique is used for decreasing overfitting thereby incresing network accuracy.

In [ ]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Flatten(input_shape=(78,)),
    tf.keras.layers.Dense(67, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(15, activation='softmax')
])

For learning rate optimization we used Adam optimizer.
Loss function used is sparse categorical crossentropy, which is standard for multiclass classification problems.

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
import os

In the next cell we setup training logs for tensorboard as well as some tensorboard callbacks.

* tensorboard - callback that logs training data.
* EarlyStopping - callback that monitors 'loss (function)' metric and if the loss function does not get better in tne hext 10 iterations, callback stops the training and resotres the network with best weights up untill that iteration.

In [ ]:
log_dir = os.path.join(
    "train_logs",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)

# TF callback that sets up TensorBoard with training logs.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# TF callback that stops training when best value of validationi loss function is reached. It also
# restores weights from the best training iteration.
eary_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

In [ ]:
model.fit(features_train,
          labels_train,
          epochs=100,
          callbacks=[tensorboard_callback, eary_stop_callback])

We can see that training stoped after 18 out of 100 epochs due to 'loss' function metric not changing much in the previous 10 epochs.

After training we evaluate model accuracy (next cell), and find that our model predicts attacks with **91.2% accuracy**.

In [ ]:
# Evaluating model accuracy.
model.evaluate(features_test, labels_test, verbose=2)

In [ ]:
# Saving the model.

model.save('saved_models/IDS_model_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.h5')

# Conclusion

In this project we made a neural network classifier that can predict 14 network/web attacks and normal traffic with 91% accuracy. This model is proof-of-concept that feedforward neural network with backpropagation algorithm can be used for classifying attacks in anomaly-based intrusion detection systems.


**Propositions**

We propose a couple of solutions for improving model accuracy as well as use of some other neural network architectures.

Accuracy of this model can probably be improved by _feature engineering_ and _feature selection_. Picking the features that have the most influence on the model.

Regarding this model, we propose tuning the model hyperparameters. Changing the hidden layer activation function, early stopping callback, dropout, optimizer and loss function should increase accuracy by some extent. Another way, albeit more complicated and resource intense is to use a genetic algorithm to evolve the best neural network arhitecture for this specific task.

Finally, we propose the usage of some other ML algorithms. Random forest classifiers have been used in intrusion detection system for a while now. Alternatively, we found some sources using autoencoders for anomaly detection.